# Analysing the redundancies of User Stories - Annotations but no US Text

In [ ]:
import locale
import os, json
from itertools import combinations

import pandas as pd
import tiktoken
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter as utils_get_column_letter

from prompt_structure.helper_prompt_composition import PromptHelperBuilder
from prompt_structure.prompt_builder import PromptBuilder

from support_functions.excel_helper import save_to_excel, formatter_ignored_items
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.request_handler import process_user_stories, process_user_stories_parallel
from support_functions.data_transformations import convert_annotation_dataset
from support_functions.time_recorder import TimeRecorder
from support_functions.json_validator import validation, chat_gpt_schema_with_annotations

: 

: 

In [ ]:
locale.setlocale(locale.LC_ALL, 'de_DE') 
MODEL_VERSION_NAME = os.getenv('MODEL_VERSION')
THREADING = bool(int(os.getenv('THREADING')))
print(f"Threading is used: {THREADING}")
print(MODEL_VERSION_NAME)

Threading is used: True
gpt-3.5-turbo


: 

In [ ]:
time_recorders: dict = {}
time_recorder: TimeRecorder = None

: 

### Load all User Stories from the different packages

We use here just the annotations (formatted):
- Triggers
- Targets
- Contains

We do not need:
- POS

Ignored Files as they differer from the common structure [Ref](https://github.com/ace-design/nlp-stories/tree/main?tab=readme-ov-file#note):
- g02-federal-funding
- g13-planningpoker
- g17-cask
- g27-culrepo

Ignored Files as they were used as trainings data
- g05
- g12

Data sets to analyse:
 - 'g03', 'g04', 'g08', 'g10', 'g11', 'g14', 'g16', 'g18', 'g19', 'g21', 'g22', 'g23', 'g24', 'g25', 'g26', 'g28'

Used Data:
- "nlp" --> "nlp_outputs" --> "individual_backlog" --> "nlp_outputs_original" --> "pos_baseline"

Highlighting words with #

In [ ]:
not_processed_datasets: dict[str, list] = loading()
print(not_processed_datasets["g02"][0])

{'PID': '#G02#', 'Text': '#G02# As a Data user, I want to have the 12-19-2017 deletions processed.', 'Persona': ['Data user'], 'Action': {'Primary Action': ['have'], 'Secondary Action': ['processed']}, 'Entity': {'Primary Entity': ['12-19-2017 deletions'], 'Secondary Entity': ['']}, 'Benefit': '', 'Triggers': [['Data user', 'have']], 'Targets': [['processed', '12-19-2017 deletions'], ['have', '12-19-2017 deletions']], 'Contains': [], 'Persona POS': {'Persona POS tag': [['NOUN', 'NOUN']], 'Persona POS text': [['Data', 'user']]}, 'Action POS': {'Primary Action POS': {'Primary Action POS tag': [['VERB']], 'Primary Action POS text': [['have']]}, 'Secondary Action POS': {'Secondary Action POS tag': [['VERB']], 'Secondary Action POS text': [['processed']]}}, 'Entity POS': {'Primary Entity POS': {'Primary Entity POS tag': [['NUM', 'NOUN']], 'Primary Entity POS text': [['12-19-2017', 'deletions']]}, 'Secondary Entity POS': {'Secondary Entity POS tag': [[]], 'Secondary Entity POS text': [[]]}}}

: 

### Converting Data to used suited format

In [ ]:
datasets, ignored_items = convert_annotation_dataset(not_processed_datasets)

: 

In [ ]:
datasets.keys()

dict_keys(['g02', 'g03', 'g04', 'g05', 'g08', 'g10', 'g11', 'g12', 'g13', 'g14', 'g16', 'g17', 'g18', 'g19', 'g21', 'g22', 'g23', 'g24', 'g25', 'g26', 'g27', 'g28'])

: 

In [ ]:
datasets["g05"]

[{'PID': '#G05#',
  'USID': '373',
  'Text': 'As a Data Publishing User, I want to be able to edit a dataset I have published, So that I can correct or enhance existing data.',
  'Main Part': 'As a Data Publishing User, I want to be able to edit a dataset I have published',
  'Benefit': 'I can correct or enhance existing data',
  'Triggers': {'Main Part': [['Data Publishing User', 'edit']], 'Benefit': []},
  'Targets': {'Main Part': [['edit', 'dataset I have published']],
   'Benefit': [['correct', 'existing data'], ['enhance', 'existing data']]},
  'Contains': {'Main Part': [], 'Benefit': []}},
 {'PID': '#G05#',
  'USID': '374',
  'Text': 'As a Data Publishing User, I want to be able to edit the model of data I have already imported, So that I can fix bugs or make enhancements in the API built for my data.',
  'Main Part': 'As a Data Publishing User, I want to be able to edit the model of data I have already imported',
  'Benefit': 'I can fix bugs or make enhancements in the API built

: 

In [ ]:
ignored_items["#G02#"][0]

'PID: #G02#; Text: As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.; Label Type: Contain'

: 

In [ ]:
df_ignored_items: pd.DataFrame = None
if True:            
    cols: list[str] =  ["PID", "Text", "Label Type"]
    df_ignored_items = pd.DataFrame(columns=cols)
    current_ignored_data: dict = {}
    curremt_pid: int = 0
    current_text: str = "" 
    for key, set in ignored_items.items():
        for item in set:
            parts = item.split('; ')
            for part in parts:
                key, value = part.split(': ', 1)
                current_ignored_data[key] = value
            df_ignored_items = pd.concat([df_ignored_items, pd.DataFrame([current_ignored_data])], ignore_index = True)               
            df_ignored_items.reset_index()
    save_to_excel(df_ignored_items, formatter_ignored_items, "Incomplete US labelling", os.getenv("OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS"))
print(f"Count of ignored Elements: {df_ignored_items.shape[0]}")
df_ignored_items.head()

Count of ignored Elements: 195


,PID,Text,Label Type
0,#G02#,"As a UI designer, I want to redesign the Resou...",Contain
1,#G02#,"As a UI designer, I want to track the issues t...",Target
2,#G02#,"As a UI designer, I want to track the issues t...",Target
3,#G02#,"As a UI designer, I want to track the issues t...",Target
4,#G02#,"As an agency user, I want to submit my data el...",Contain


: 

### Datatransformation to panda dataframes and numpy arrarys. 

In [ ]:
def transform_2_dataframe(data: list):
    rows = []
    for entry in data:
        row = {
            "PID": entry["PID"],
            "USID": entry["USID"],
            "Text": entry["Text"],
            "Main Part": entry["Main Part"],
            "Benefit": entry["Benefit"],
            "Triggers": entry["Triggers"],
            "Targets": entry["Targets"],
            "Contains": entry["Contains"],
        }
        rows.append(row)
    return pd.DataFrame(rows)

: 

In [ ]:
# n(n-1) / 2
def transform_pairwise(df: pd.DataFrame):
    rows = []
    for i, j in combinations(range(df.shape[0]), 2):
        row = {
            "First PID": df.iloc[i, 0],
            "First USID": df.iloc[i, 1],
            "First Text": df.iloc[i, 2],
            "First Main Part": df.iloc[i, 3],
            "First Benefit": df.iloc[i, 4],
            "First Triggers": df.iloc[i, 5],
            "First Targets": df.iloc[i, 6],
            "First Contains": df.iloc[i, 7],
            "Second PID": df.iloc[j, 0],
            "Second USID": df.iloc[j, 1],
            "Second Text": df.iloc[j, 2],
            "Second Main Part": df.iloc[j, 3],
            "Second Benefit": df.iloc[j, 4],
            "Second Triggers": df.iloc[j, 5],
            "Second Targets": df.iloc[j, 6],
            "Second Contains": df.iloc[j, 7]
        }
        rows.append(row)
    return pd.DataFrame(rows)

: 

In [ ]:
def convert_dataframe_to_json(pair: pd.Series) -> tuple[dict, dict]:
    pair = pair.to_dict()
    us_one: dict = {
        "USID": pair["First USID"],
        "Triggers": pair["First Triggers"],
        "Targets": pair["First Targets"],
        "Contains": pair["First Contains"]
    }
    
    us_two: dict = {
        "USID": pair["Second USID"],
        "Triggers": pair["Second Triggers"],
        "Targets": pair["Second Targets"],
        "Contains": pair["Second Contains"]
    }

    json_one: dict = json.loads(json.dumps(us_one))
    json_two: dict = json.loads(json.dumps(us_two))
    
    return json_one, json_two

: 

In [ ]:
def template_request_two_user_stories(
    current_message: list[dict],
    idx: int,
    pairs: pd.DataFrame,
) -> None:
    row = pairs.iloc[idx]
    json_us_one: dict = None
    json_us_two: dict = None
    json_us_one, json_us_two = convert_dataframe_to_json(row)
    request: dict = PromptHelperBuilder.get_instance().parsing_to_pair_requests(json_us_one, json_us_two)
    current_message.append(request)

: 

In [ ]:
def sort_threaded_results(to_sort: dict) -> None:
    new_list: list = None
    for _ in to_sort.values():
        new_list = sorted(_, key=lambda x: (int(x['relatedStories'][0]), int(x['relatedStories'][1])))
        _.clear()
        _ += new_list

: 

In [ ]:
def json_validator(json_data: dict) -> tuple[bool, str]:
    return validation(json_data, chat_gpt_schema_with_annotations)

: 

### Prepaire Prompting

In [ ]:
# Init PromptBuilder
builder: PromptBuilder = PromptBuilder.get_instance()

# Collection of messages
message: list[dict] = []

: 

In [ ]:
# Actor Role
message.append(builder.get_actor_role())
message.append(builder.get_system_simulation_actor_role())

: 

In [ ]:
# User Story Definition
message.append(builder.get_user_story_definition())
message.append(builder.get_system_simulation_user_story_definition())

: 

In [ ]:
# Task and fucos definition
message.append(builder.get_task_focus_definition())
message.append(builder.get_system_simulation_task_focus_definition())

: 

In [ ]:
# Redundancy Definition for User Story pair
message.append(builder.get_redundancy_full_partial_definition())
message.append(builder.get_system_simulation_redundancy_full_partial_definition())

: 

In [ ]:
# Defining the JSON output
message.append(builder.get_json_format_defintion())
message.append(builder.get_system_simulation_json_format_defintion())

: 

In [ ]:
# Providing Examples
message.append(builder.get_input_output_examples(["USID", "Triggers", "Targets", "Contains"]))
message.append(builder.get_system_simulation_example_consideration())

: 

In [ ]:
if True:
    message_text: str = ""
    for key in message:
        message_text += key["content"]
        print(key["content"])
    enc = tiktoken.get_encoding('cl100k_base')
    token_size = enc.encode(message_text)
    print("-" * 3 + "Token" + "-" * 3)
    print(token_size)
    print("-" * 3 + "Sum of Tokens" + "-" * 3)
    # The output is not correct as the result from the online pages differs https://platform.openai.com/tokenizer
    print("The total token sum is: " + locale.format_string("%d", sum(token_size), grouping=True))
    

: 

: 

## Data processing for: G03

In [ ]:
ID_G03: str = "g03"
data_g03: list = datasets[ID_G03]
df_g03: pd.DataFrame = transform_2_dataframe(data_g03)
df_g03_pairs = transform_pairwise(df_g03)

In [ ]:
datasets[ID_G03][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g03.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g03_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G03] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g03_pairs, key=ID_G03, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g03_pairs, key=ID_G03, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G04

In [ ]:
ID_G04: str = "g04"
data_g04: list = datasets[ID_G04]
df_g04: pd.DataFrame = transform_2_dataframe(data_g04)
df_g04_pairs = transform_pairwise(df_g04)

In [ ]:
datasets[ID_G04][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g04.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g04_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G04] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g04_pairs, key=ID_G04, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g04_pairs, key=ID_G04, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G08

In [ ]:
ID_G08: str = "g08"
data_g08: list = datasets[ID_G08]
df_g08: pd.DataFrame = transform_2_dataframe(data_g08)
df_g08_pairs = transform_pairwise(df_g08)

In [ ]:
datasets[ID_G08][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g08.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g08_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G08] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g08_pairs, key=ID_G08, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g08_pairs, key=ID_G08, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G10

In [ ]:
ID_G10: str = "g10"
data_g10: list = datasets[ID_G10]
df_g10: pd.DataFrame = transform_2_dataframe(data_g10)
df_g10_pairs = transform_pairwise(df_g10)

In [ ]:
datasets[ID_G10][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g10.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g10_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G10] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g10_pairs, key=ID_G10, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g10_pairs, key=ID_G10, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G11

In [ ]:
ID_G11: str = "g11"
data_g11: list = datasets[ID_G11]
df_g11: pd.DataFrame = transform_2_dataframe(data_g11)
df_g11_pairs = transform_pairwise(df_g11)

In [ ]:
datasets[ID_G11][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g11.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g11_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G11] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g11_pairs, key=ID_G11, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g11_pairs, key=ID_G11, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G14

In [ ]:
ID_G14: str = "g14"
data_g14: list = datasets[ID_G14]
df_g14: pd.DataFrame = transform_2_dataframe(data_g14)
df_g14_pairs = transform_pairwise(df_g14)

In [ ]:
datasets[ID_G14][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g14.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g14_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G14] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g14_pairs, key=ID_G14, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g14_pairs, key=ID_G14, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G16

In [ ]:
ID_G16: str = "g16"
data_g16: list = datasets[ID_G16]
df_g16: pd.DataFrame = transform_2_dataframe(data_g16)
df_g16_pairs = transform_pairwise(df_g16)

In [ ]:
datasets[ID_G16][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g16.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g16_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G16] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g16_pairs, key=ID_G16, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g16_pairs, key=ID_G16, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G18

In [ ]:
ID_G18: str = "g18"
data_g18: list = datasets[ID_G18]
df_g18: pd.DataFrame = transform_2_dataframe(data_g18)
df_g18_pairs = transform_pairwise(df_g18)

In [ ]:
datasets[ID_G18][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g18.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g18_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G18] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g18_pairs, key=ID_G18, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g18_pairs, key=ID_G18, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G19

In [ ]:
ID_G19: str = "g19"
data_g19: list = datasets[ID_G19]
df_g19: pd.DataFrame = transform_2_dataframe(data_g19)
df_g19_pairs = transform_pairwise(df_g19)

In [ ]:
datasets[ID_G19][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g19.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g19_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G19] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g19_pairs, key=ID_G19, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g19_pairs, key=ID_G19, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G21

In [ ]:
ID_G21: str = "g21"
data_g21: list = datasets[ID_G21]
df_g21: pd.DataFrame = transform_2_dataframe(data_g21)
df_g21_pairs = transform_pairwise(df_g21)

In [ ]:
datasets[ID_G21][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g21.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g21_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G21] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g21_pairs, key=ID_G21, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g21_pairs, key=ID_G21, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G22

In [ ]:
ID_G22: str = "g22"
data_g22: list = datasets[ID_G22]
df_g22: pd.DataFrame = transform_2_dataframe(data_g22)
df_g22_pairs = transform_pairwise(df_g22)

In [ ]:
datasets[ID_G22][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g22.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g22_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G22] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g22_pairs, key=ID_G22, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g22_pairs, key=ID_G22, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G23

In [ ]:
ID_G23: str = "g23"
data_g23: list = datasets[ID_G23]
df_g23: pd.DataFrame = transform_2_dataframe(data_g23)
df_g23_pairs = transform_pairwise(df_g23)

In [ ]:
datasets[ID_G23][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g23.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g23_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G23] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g23_pairs, key=ID_G23, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g23_pairs, key=ID_G23, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G24

In [ ]:
ID_G24: str = "g24"
data_g24: list = datasets[ID_G24]
df_g24: pd.DataFrame = transform_2_dataframe(data_g24)
df_g24_pairs = transform_pairwise(df_g24)

In [ ]:
datasets[ID_G24][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g24.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g24_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G24] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g24_pairs, key=ID_G24, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g24_pairs, key=ID_G24, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G25

In [ ]:
ID_G25: str = "g25"
data_g25: list = datasets[ID_G25]
df_g25: pd.DataFrame = transform_2_dataframe(data_g25)
df_g25_pairs = transform_pairwise(df_g25)

In [ ]:
datasets[ID_G25][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g25.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g25_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G25] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g25_pairs, key=ID_G25, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g25_pairs, key=ID_G25, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G26

In [ ]:
ID_G26: str = "g26"
data_g26: list = datasets[ID_G26]
df_g26: pd.DataFrame = transform_2_dataframe(data_g26)
df_g26_pairs = transform_pairwise(df_g26)

In [ ]:
datasets[ID_G26][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g26.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g26_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G26] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g26_pairs, key=ID_G26, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g26_pairs, key=ID_G26, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G27

In [ ]:
ID_G27: str = "g27"
data_g27: list = datasets[ID_G27]
df_g27: pd.DataFrame = transform_2_dataframe(data_g27)
df_g27_pairs = transform_pairwise(df_g27)

In [ ]:
datasets[ID_G27][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g27.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g27_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G27] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g27_pairs, key=ID_G27, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g27_pairs, key=ID_G27, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G28

In [ ]:
ID_G28: str = "g28"
data_g28: list = datasets[ID_G28]
df_g28: pd.DataFrame = transform_2_dataframe(data_g28)
df_g28_pairs = transform_pairwise(df_g28)

In [ ]:
datasets[ID_G28][0]

{'PID': '#G03#',
 'USID': '315',
 'Text': 'As a Public User, I want to Search for Information, so that I can obtain publicly available information concerning properties, County services, processes and other general information.',
 'Main Part': 'As a Public User, I want to Search for Information, ',
 'Benefit': 'I can obtain publicly available information concerning properties, County services, processes and other general information',
 'Triggers': {'Main Part': [['Public User', 'Search']], 'Benefit': []},
 'Targets': {'Main Part': [['Search', 'Information']],
  'Benefit': [['obtain', 'publicly available information']]},
 'Contains': {'Main Part': [],
  'Benefit': [['publicly available information', 'properties'],
   ['publicly available information', 'County services'],
   ['publicly available information', 'processes'],
   ['publicly available information', 'other general information']]}}

In [ ]:
display(df_g28.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail..."
1,#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
2,#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
3,#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
4,#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"


In [ ]:
display(df_g28_pairs.head())

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,316,"As a ProspectiveApplicant, I want to research ...","As a ProspectiveApplicant, I want to research ...",I can find the relevant service and/or applica...,"{'Main Part': [['ProspectiveApplicant', 'resea...","{'Main Part': [['research', 'requirements'], [...","{'Main Part': [], 'Benefit': [['online portal'..."
1,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,317,"As an Applicant, I want to Request PreApplicat...","As an Applicant, I want to Request PreApplicat...",I can receive a response to a request for a me...,"{'Main Part': [['Applicant', 'Request']], 'Ben...","{'Main Part': [['Request', 'PreApplication Ass...","{'Main Part': [], 'Benefit': [['request for a ..."
2,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,318,"As a Customer, I want to Create a Customer Por...","As a Customer, I want to Create a Customer Por...",I can log on to the Customer Portal and perfor...,"{'Main Part': [['Customer', 'Create']], 'Benef...","{'Main Part': [['Create', 'Customer Portal Use...","{'Main Part': [['Customer Portal', 'Customer P..."
3,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,319,"As an Applicant, I want to Submit Application,...","As an Applicant, I want to Submit Application,","I can provide my information, plans and/or doc...","{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Application']], 'Be...","{'Main Part': [], 'Benefit': []}"
4,#G03#,315,"As a Public User, I want to Search for Informa...","As a Public User, I want to Search for Informa...",I can obtain publicly available information co...,"{'Main Part': [['Public User', 'Search']], 'Be...","{'Main Part': [['Search', 'Information']], 'Be...","{'Main Part': [], 'Benefit': [['publicly avail...",#G03#,320,"As an Applicant, I want to Submit Supporting D...","As an Applicant, I want to Submit Supporting D...",I can satisfy documentation requirements for m...,"{'Main Part': [['Applicant', 'Submit']], 'Bene...","{'Main Part': [['Submit', 'Supporting Document...","{'Main Part': [], 'Benefit': [['application', ..."


### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G28] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=2, message=message, pairs=df_g28_pairs, key=ID_G28, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=2, message=message, pairs=df_g28_pairs, key=ID_G28, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Save total Speeds

In [ ]:
SHEET_NAME = 'Time Consup. Anlys.S.'

In [ ]:
base_path: str = os.getcwd()
path_to_file = os.path.join(base_path, "results")
path_to_file = os.path.join(path_to_file, "redundancy-model-" + MODEL_VERSION_NAME)

count_runs_per_data_set: dict[str, int] = {}
for key in time_recorders.keys():
        idx = 0
        _ =  f"{idx:02d}_{key}.json"
        while os.path.exists(os.path.join(path_to_file, f"{idx:02d}_{key}.json")):
                idx += 1
        count_runs_per_data_set[key] = idx

entries: list[tuple[int,str,str, int, float, float, float]] = []

entry: tuple[int,str,str, int, float, float, float] = None
for key, value in time_recorders.items():
        entry = (key, count_runs_per_data_set[key], MODEL_VERSION_NAME, str(THREADING).lower(), 
                 value.nanoseconds, value.milliseconds, value.seconds, value.minutes)
        entries.append(entry)

columns = ['Dataset', 'Run Count', 'Model Version', 'Threading Enabled', 
           'Nanoseconds', 'Milliseconds', 'Seconds', 'Minutes']

_file_path: str = os.path.join(base_path, os.getenv('OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS_MODIFIED'))
check = os.path.exists(_file_path)

old_time_consuption_data: pd.DataFrame = None
if check:
        try:
                old_time_consuption_data = pd.read_excel(_file_path, SHEET_NAME)
        except ValueError:
                check = False
           
time_consumption_data: pd.DataFrame = pd.DataFrame(entries, columns=columns)

if check and not old_time_consuption_data.empty:
        # Checking if in the excel is already the data. Case: this code is executed twice for the same data
        old_time_consuption_data = old_time_consuption_data.dropna()
        for idx in range(len(old_time_consuption_data)):
                condition = time_consumption_data[
                        (time_consumption_data['Dataset'] == old_time_consuption_data.iat[idx, 0]) & 
                        (time_consumption_data['Run Count'] == old_time_consuption_data.iat[idx, 1])
                ].index
                time_consumption_data = time_consumption_data.drop(condition).reset_index(drop=True)
        time_consumption_data = pd.concat([old_time_consuption_data, time_consumption_data]).reset_index(drop=True)

In [ ]:
def formatter_time(wb: Workbook, sheet_name: str): 
    ws = wb[sheet_name]
    header_font = Font(size=14, bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
    
    ADDITIONAL_LENGTH: int = 0
    ADJUSTED_WIDTH: int = 0
    MAX_LEN: int = 0
    for col in ws.iter_cols(min_row=1, max_row=1):
        for cell in col:
            MAX_LEN = len(str(cell.value))
            ADDITIONAL_LENGTH = (MAX_LEN + 2)
            ADJUSTED_WIDTH = 0
            ADJUSTED_WIDTH =  ADDITIONAL_LENGTH * 1.5
            ws.column_dimensions[utils_get_column_letter(cell.column)].width = ADJUSTED_WIDTH
            
    alignment = Alignment(vertical='center', horizontal='left')
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = alignment

    num_columns = ws.max_column
    header_range = f"A1:{utils_get_column_letter(num_columns)}1"
    ws.auto_filter.ref = header_range
    ws.freeze_panes = ws['A2']

    wrap_alignment = Alignment(wrap_text=True, vertical='top', horizontal='left')
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            cell.alignment = wrap_alignment

In [ ]:
save_to_excel(time_consumption_data, formatter_time, SHEET_NAME, os.getenv("OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS"))